In [1]:
import copy
import pandas as pd
import pickle
from flaml import AutoML
from sklearn.model_selection import train_test_split
import numpy as np
import pandas_ta as ta

class DomesticIndexPreprocessor:
    def __init__(self, file_path, target_idx_code):
        self.file_path = file_path
        self.target_idx_code = target_idx_code
        self.df = pd.read_excel(file_path, dtype={'idxCode': str})
        self.result_data = {}

    def preprocess(self, df, targetIdxCode):
        code = df.idxCode.unique()[0]

        # 데이터 로드
        data = copy.deepcopy(df)

        # 날짜 기준 정렬
        data = data.sort_values("stck_bsop_date").reset_index(drop=True)

        # ------------------------------------
        # 기초 통계
        # ------------------------------------
        
        # 등락률 계산 (전일 대비)
        ## 전일 대비 비율 변화를 계산하여 추세를 파악
        pct_change_cols = ["bstp_nmix_prpr", "bstp_nmix_oprc", "bstp_nmix_hgpr", "bstp_nmix_lwpr", "acml_vol", "acml_tr_pbmn"]
        for col in pct_change_cols:
            data[f"{col}_pct_change"] = data[col].pct_change()
        
        # 변동폭 (고가 - 저가)
        ## 하루 동안의 주가 변동 범위를 측정하여 변동성을 파악
        data["range"] = data["bstp_nmix_hgpr"] - data["bstp_nmix_lwpr"]
        
        # 저가대비 고가 비율 (고가 / 저가)
        ## 하루 동안의 고가와 저가의 비율을 계산하여 상대적 주가 변동 정도를 평가
        data["price_ratio"] = data["bstp_nmix_hgpr"] / data["bstp_nmix_lwpr"]
        
        # 변동률 (변동폭 / 시가)
        ## 변동폭을 시가로 나누어 하루 주가 변동성을 백분율로 측정
        data["range_pct"] = data["range"] / data["bstp_nmix_oprc"]
        
        # 거래대금 대비 거래량 (누적거래대금 / 거래량)
        ## 하루 동안의 거래대금과 거래량의 비율을 계산하여 거래의 밀도를 평가
        data["vol_to_tr_pbmn_ratio"] = data["acml_tr_pbmn"] / data["acml_vol"]
        
        # Volume Spike: 거래량의 급증 정도를 나타내는 z-score
        ## 평균 거래량 대비 현재 거래량의 급증 정도를 정규화하여 평가
        data["volume_spike"] = (data['acml_vol'] - data['acml_vol'].mean()) / data['acml_vol'].std()
        
        # 누적 수익률 계산 (Daily_Return 컬럼이 존재한다고 가정)
        ## 일별 등락률을 기준으로 누적 수익률을 계산하여 장기적 투자 성과를 평가
        data['cumulative_return'] = (1 + data['bstp_nmix_prpr'].pct_change()).cumprod()
        
        # 이동 변동성 계산 (5일 기준)
        ## 5일 동안의 주가 변동률의 표준편차를 계산하여 단기 변동성을 평가
        rolling_window = 5
        data['rolling_volatility'] = data['bstp_nmix_prpr'].pct_change().rolling(window=rolling_window).std()
        
        # 돌파 지표 계산 (최근 n일 동안 최고치 돌파 여부)
        ## 고가가 최근 n일 최고치를 돌파했는지 여부를 나타냄 (1: 돌파, 0: 미돌파)
        n = 20
        data['breakout_indicator'] = np.where(
            data['bstp_nmix_hgpr'] > data['bstp_nmix_hgpr'].rolling(window=n).max(), 1, 0
        )
        
        # 저항선 (최근 n일 동안의 최고치)
        ## 주가가 돌파해야 할 저항선 수준을 계산
        data['resistance_level'] = data['bstp_nmix_hgpr'].rolling(window=n).max()
        
        # 지지선 (최근 n일 동안의 최저치)
        ## 주가가 하락하지 않을 것으로 예상되는 지지선 수준을 계산
        data['support_level'] = data['bstp_nmix_lwpr'].rolling(window=n).min()
        
        # ------------------------------------
        # pandas_ta 기반 기술적 지표 계산
        # !!!!! pandas_ta 사전 설치 필요 !!!!!
        # ------------------------------------
        
        # 단순 이동평균선 (SMA)
        # 일정 기간 동안의 주가 평균을 계산하여 추세를 확인
        data['SMA_5'] = ta.sma(data['bstp_nmix_prpr'], length=5)
        data['SMA_20'] = ta.sma(data['bstp_nmix_prpr'], length=20)
        data['SMA_50'] = ta.sma(data['bstp_nmix_prpr'], length=50)
        
        # 지수 이동평균선 (EMA)
        # 최근 데이터에 더 높은 가중치를 두어 추세를 확인
        data['EMA_5'] = ta.ema(data['bstp_nmix_prpr'], length=5)
        data['EMA_20'] = ta.ema(data['bstp_nmix_prpr'], length=20)
        data['EMA_50'] = ta.ema(data['bstp_nmix_prpr'], length=50)
        
        # RSI (상대강도지수)
        # 과매수(70 이상) 또는 과매도(30 이하) 여부를 평가
        data['RSI_14'] = ta.rsi(data['bstp_nmix_prpr'], length=14)
        data['RSI_30'] = ta.rsi(data['bstp_nmix_prpr'], length=30)
        
        # MACD (이동평균 수렴·발산)
        # 단기 이동평균선과 장기 이동평균선의 차이를 통해 추세 전환을 탐지
        macd = ta.macd(data['bstp_nmix_prpr'], fast=12, slow=26, signal=9)
        data['MACD'] = macd['MACD_12_26_9']
        data['MACD_signal'] = macd['MACDs_12_26_9']
        data['MACD_hist'] = macd['MACDh_12_26_9']
        
        # 볼린저 밴드 (Bollinger Bands)
        # 주가가 이동평균선 대비 얼마나 벗어났는지를 평가 (상단, 하단 밴드 포함)
        bb = ta.bbands(data['bstp_nmix_prpr'], length=20, std=2)
        data['BB_upper'] = bb['BBU_20_2.0']
        data['BB_middle'] = bb['BBM_20_2.0']
        data['BB_lower'] = bb['BBL_20_2.0']
        
        # ATR (Average True Range)
        # 주가의 변동 폭(진폭)을 계산하여 변동성을 평가
        data['ATR_14'] = ta.atr(data['bstp_nmix_hgpr'], data['bstp_nmix_lwpr'], data['bstp_nmix_prpr'], length=14)
        
        # CCI (Commodity Channel Index)
        # 평균 가격과 현재 가격 간의 차이를 기준으로 과매수/과매도 여부를 평가
        data['CCI_20'] = ta.cci(data['bstp_nmix_hgpr'], data['bstp_nmix_lwpr'], data['bstp_nmix_prpr'], length=20)
        
        # ADX (Average Directional Index)
        # 추세 강도를 측정하며 25 이상이면 강한 추세로 간주
        adx = ta.adx(data['bstp_nmix_hgpr'], data['bstp_nmix_lwpr'], data['bstp_nmix_prpr'], length=14)
        data['ADX_14'] = adx['ADX_14']
        
        # 스토캐스틱 오실레이터
        # 현재가가 일정 기간의 고가/저가 범위 내 어디에 위치하는지를 평가
        stoch = ta.stoch(data['bstp_nmix_hgpr'], data['bstp_nmix_lwpr'], data['bstp_nmix_prpr'], k=14, d=3)
        data['Stoch_%K'] = stoch['STOCHk_14_3_3']
        data['Stoch_%D'] = stoch['STOCHd_14_3_3']
        
        # 모멘텀 (Momentum)
        # 일정 기간 동안의 주가 변화량을 계산하여 추세 강도를 평가
        data['Momentum_10'] = data['bstp_nmix_prpr'] - data['bstp_nmix_prpr'].shift(10)
        
        # Williams %R
        # 주가가 최근 고가/저가 대비 어느 위치에 있는지 계산 (과매수/과매도 확인)
        data['Williams_%R'] = ta.willr(data['bstp_nmix_hgpr'], data['bstp_nmix_lwpr'], data['bstp_nmix_prpr'], length=14)
        
        # 이동평균 크로스오버
        # 단기 이동평균선과 장기 이동평균선의 교차를 통해 골든 크로스/데드 크로스를 탐지
        data['MA_Crossover'] = np.where(data['SMA_20'] > data['SMA_50'], 1, -1)  # 1: 골든 크로스, -1: 데드 크로스
        
        # 타겟값 생성 (전일 대비 상승: 1, 하락: 0)
        target_cond = code == targetIdxCode
        if target_cond:
            data["target"] = (data["bstp_nmix_prpr"].diff() > 0).astype(int)

        # 컬럼 이름에 지수 코드 추가
        data.columns = [x + '_' + code  if x not in ['target', 'stck_bsop_date'] else x for x in data.columns]

        return data

    def process_all_data(self):
        loop_code = self.df.idxCode.unique()

        for idxCode in loop_code:
            temp_df = self.df.loc[self.df.idxCode == idxCode]
            if idxCode == loop_code[0]:
                data = self.preprocess(temp_df, self.target_idx_code)
            else:
                temp_data = self.preprocess(temp_df, self.target_idx_code)
                data = pd.merge(data, temp_data, on='stck_bsop_date', how='left')

        # 마지막 행 분리 (예측에 사용할 데이터)
        X_last = data.iloc[[-1]].drop(columns=["target"])  # 마지막 행에서 타겟값 제외
        data = data.iloc[:-1]  # 마지막 행 제외한 데이터로 학습

        # 입력 변수(X)와 타겟 변수(y) 분리
        X = data.drop(columns=["target"])
        X = X.replace([float("inf"), float("-inf")], pd.NA)
        y = data["target"]

        # 결과물을 딕셔너리에 담아서 저장
        self.result_data = {
            "X_last": X_last,
            "X": X,
            "y": y
        }

    def save_results(self):
        # 저장되는 파일 이름 생성
        file_name = "ds_domesticIndex_flaml.pkl"
        
        # pickle로 저장
        with open(file_name, 'wb') as f:
            pickle.dump(self.result_data, f)
            
        print(f"파일 저장 완료: {file_name}")

# 사용 예시
file_path = 'domesticIndex.xlsx'
target_idx_code = '0001'

# 클래스 인스턴스 생성
preprocessor = DomesticIndexPreprocessor(file_path, target_idx_code)

# 데이터 처리
preprocessor.process_all_data()

# 결과 저장
preprocessor.save_results()

파일 저장 완료: ds_domesticIndex_flaml.pkl


In [2]:
preprocessor.result_data['X_last']

,stck_bsop_date,bstp_nmix_prpr_0001,prdy_vrss_sign_0001,bstp_nmix_prdy_vrss_0001,bstp_nmix_prdy_ctrt_0001,bstp_nmix_oprc_0001,bstp_nmix_hgpr_0001,bstp_nmix_lwpr_0001,acml_vol_rlim_0001,acml_vol_0001,...,BB_middle_0005,BB_lower_0005,ATR_14_0005,CCI_20_0005,ADX_14_0005,Stoch_%K_0005,Stoch_%D_0005,Momentum_10_0005,Williams_%R_0005,MA_Crossover_0005
999,2024-12-17,2464.18,5,-24.79,-1.0,2487.31,2487.31,2464.18,100.0,408747,...,3774.6105,3676.542933,71.455989,68.380006,14.59225,75.023463,80.623236,-20.93,-44.18284,1


In [3]:
preprocessor.result_data['X']

,stck_bsop_date,bstp_nmix_prpr_0001,prdy_vrss_sign_0001,bstp_nmix_prdy_vrss_0001,bstp_nmix_prdy_ctrt_0001,bstp_nmix_oprc_0001,bstp_nmix_hgpr_0001,bstp_nmix_lwpr_0001,acml_vol_rlim_0001,acml_vol_0001,...,BB_middle_0005,BB_lower_0005,ATR_14_0005,CCI_20_0005,ADX_14_0005,Stoch_%K_0005,Stoch_%D_0005,Momentum_10_0005,Williams_%R_0005,MA_Crossover_0005
0,2020-11-25,2601.54,5,-16.22,-0.62,2637.34,2642.26,2583.41,28.99,1410105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,2020-11-26,2625.91,2,24.37,0.94,2605.73,2625.97,2592.43,29.46,1387435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,2020-11-27,2633.45,2,7.54,0.29,2624.80,2635.00,2618.47,34.17,1196042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,2020-11-30,2591.34,5,-42.11,-1.60,2648.05,2648.66,2591.34,31.16,1311831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,2020-12-01,2634.25,2,42.91,1.66,2613.42,2638.87,2611.67,35.51,1151129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2024-12-10,2417.84,2,57.26,2.43,2384.51,2419.06,2384.51,68.26,598770,...,3722.2985,3576.495956,76.139034,74.847312,15.255524,43.201393,52.977967,71.44,-33.791461,-1
995,2024-12-11,2442.51,2,24.67,1.02,2412.15,2443.34,2411.38,66.17,617717,...,3733.8875,3596.004608,75.744817,115.872882,15.266038,53.232851,47.580128,68.25,-25.022906,-1
996,2024-12-12,2482.12,2,39.61,1.62,2456.63,2487.95,2448.76,61.54,664187,...,3747.1610,3622.610299,73.788759,130.392083,15.284179,75.193941,57.209395,72.10,-15.603812,-1
997,2024-12-13,2494.46,2,12.34,0.50,2473.75,2500.32,2470.24,70.75,577737,...,3760.2955,3652.941407,73.327419,113.031460,14.866472,82.296439,70.241077,130.22,-12.483966,-1
